In [84]:
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
#lei=law, n=article number, p=paragraph number, inciso=item, alinea=point
class Scrap:
    def __init__(self, lei, n=None, p=None, inciso=None):
        """Initializing the driver"""
        self.lei = lei
        self.n = n
        self.p = p
        
        self.art = str('Art. ' + str(self.n))
        self.lista_paragrafo = []
        self.lista_inciso = []
        
        url = 'https://www2.camara.leg.br/busca/?q=' + lei
        option = Options()
        option.headless = True
        driver = webdriver.Firefox(options=option)
        driver.get(url)
        driver.find_element_by_xpath("//div[@id='resultadoBusca']//ul//li//span//a").click()
        driver.find_element_by_xpath("//div[@class='sessao']//a").click()
        element = driver.find_element_by_xpath("//div[@class='texto']")
        self.titulo = driver.find_element_by_class_name('ementa')
        html_content = element.get_attribute('outerHTML')
        self.titulo = self.titulo.get_attribute('outerHTML')
        driver.quit()

        self.soup = BeautifulSoup(html_content, 'lxml')
        self.texto = self.soup.find_all(text=True)

        self.titulo = BeautifulSoup(self.titulo, 'lxml')
        self.titulo = self.titulo.find_all(text=True)
        #limpando
        for i in range(len(self.texto)):
            self.texto[i] = self.texto[i].replace('\n', '')
            self.texto[i] = self.texto[i].replace('\t', '')
            self.texto[i] = self.texto[i].replace('\xa0', '')
        
        
    def title(self):
        """Returns the law title""" 
        for i in range(len(self.titulo)):
            self.titulo[i] = self.titulo[i].replace('\n', '')
            self.titulo[i] = self.titulo[i].replace('\t', '')
            if '' in self.titulo:
                self.titulo.remove('')
            return(self.titulo[0])
   
    def article(self):
        pattern0 = f'(Art\. {self.n})'
        patternf = r"(\'\,)"
        self.artigo0 = re.search(pattern0, str(self.texto)).start()
        self.artigof = re.search(patternf, str(self.texto)[self.artigo0:]).start()
        self.total = self.artigo0 + self.artigof
        return(str(self.texto)[self.artigo0:(self.total)])

    def paragraph(self):
        """Must returns the requested paragraph for requested article"""
        pattern0 = f'(Art\. {self.n})'
        patternf = r"(\'\,)"
        artigo0 = re.search(pattern0, str(self.texto)).start()
        artigof = re.search(patternf, str(self.texto)[artigo0:]).start()
        pattern_paragraph0 = f'(§ {self.p})'
        pattern_paragraphf = r"(\'\,)"
        paragraph0 = re.search(pattern_paragraph0, str(self.texto)[artigof:]).start()
        paragraph0 = paragraph0 + artigof
        paragraphf = re.search(pattern_paragraphf, str(self.texto)[paragraph0:]).start()
        paragraphf = paragraphf + paragraph0
        
        paragraph_single = f'Parágrafo único.'
        paragraph_single_end = f'\n'
        article1 = f'(Art\. {self.n+1})'
        article1 = re.search(article1, str(self.texto)[artigof:])
        #agora que possuo os limites(artigo0 e article1), vou buscar pelos padrões
        paragraph_single = re.search(paragraph_single, str(self.texto)[artigo0:artigo1])
        paragraph_singlef = re.search(paragraph_single_end, str(self.texto)[artigo1: paragraph_single])
        try:
            if (f'Art. {int(self.n) +1}') in str(self.texto)[(artigo0 + artigof): paragraphf].split("',")[self.p]:
                return('Parágrafo não encontrado!')
            elif ('Parágrafo único.') in str(self.texto)[(artigo0 + artigof): paragraphf].split("',")[self.p]:
                return('Parágrafo único. É uma boa seguir por aqui!')
            else:
                return(str(self.texto)[(artigo0 + artigof): paragraphf].split("',"))[self.p]
        except:
            if (f'Art. {int(self.n) +1}') in str(self.texto)[(artigo0 + artigof): paragraphf].split("',")[self.p]:
                return('Parágrafo não encontrado!')
            else:
                return(str(self.texto)[(artigo0 + artigof):].split("',"))[self.p]        

In [85]:
teste = Scrap(lei='lei 1.060', n=4, p=1)

In [86]:
teste.title()

'Estabelece normas para a concessão de assistência judiciária aos necessitados. '

In [87]:
teste.article()

'Art. 4ºA parte, que pretender gozar os benefícios da assistência judiciária, requererá ao Juiz competente lhos conceda, mencionando, na petição, o rendimento ou vencimento que percebe e os encargos próprios e os da família. '

In [88]:
teste.paragraph()

" ' § 1º A petição será instruída por um atestado de que conste ser o requerente necessitado, não podendo pagar as despesas do processo. Êste documento será expedido, isento de selos e emolumentos, pela autoridade policial ou pelo Prefeito municipal. "

In [89]:
#parágrafo único
teste2 = Scrap(lei='lei 1.060', n=2, p=1)

In [90]:
teste2.title()

'Estabelece normas para a concessão de assistência judiciária aos necessitados. '

In [91]:
teste2.article()

'Art. 2ºGozarão dos benefícios desta Lei os nacionais ou estrangeiros residentes no país, que necessitarem recorrer à Justiça penal, civil, militar ou do trabalho. '

In [92]:
teste2.paragraph()

" ' "